In [73]:
import pandas as pd
import numpy as np

In [74]:
df = pd.read_csv("data/ROC-Story-Cloze-Val.csv")
df.shape

(1871, 8)

In [75]:
def convert_train(df):
    df1 = df[['InputStoryid', 'InputSentence1', 'InputSentence2', 'InputSentence3',
           'InputSentence4', 'RandomFifthSentenceQuiz1','AnswerRightEnding']]
    labels1 = df1['AnswerRightEnding'].values
    for i in range(len(labels1)):
        if labels1[i] == 1:
            labels1[i] = 1
        else:
            labels1[i] = 0
    df1['label'] = labels1

    df2 = df[['InputStoryid', 'InputSentence1', 'InputSentence2', 'InputSentence3',
           'InputSentence4', 'RandomFifthSentenceQuiz2','AnswerRightEnding']]
    labels2 = df2['AnswerRightEnding'].values
    for i in range(len(labels2)):
        if labels2[i] == 2:
            labels2[i] = 1
        else:
            labels2[i] = 0
    df2['label'] = labels2

    df1 = df1.drop(columns=['AnswerRightEnding'])
    df1['Quiz'] = df1['RandomFifthSentenceQuiz1']
    df1 = df1.drop(columns=['RandomFifthSentenceQuiz1'])
    df2 = df2.drop(columns=['AnswerRightEnding'])
    df2['Quiz'] = df2['RandomFifthSentenceQuiz2']
    df2 = df2.drop(columns=['RandomFifthSentenceQuiz2'])
    df3 = pd.concat([df1,df2])
    df3.reset_index(drop=True,inplace=True)
    df3 = df3.sample(frac=1)
    del df1
    del df2
    df3["Story"] = df3['InputSentence1'].str.cat(df3['InputSentence2'])
    df3["Story"] = df3['Story'].str.cat(df3['InputSentence3'])
    df3["Story"] = df3['Story'].str.cat(df3['InputSentence4'])
    df3 = df3.drop(columns=['InputSentence1','InputSentence2','InputSentence3','InputSentence4'])
    df3 = df3[['InputStoryid','Story','Quiz','label']]
    return df3

In [76]:
df_train = df.head(1497)
df_dev_test = df.tail(374)
df_train = convert_train(df_train)
df_dev = convert_train(df_dev_test)
df_train.to_csv("train.tsv",sep="\t",header=None,index=None)
df_dev.to_csv("dev.tsv",sep="\t",header=None,index=None)
# df_dev.to_csv("test.tsv",sep="\t",index=None)

In [77]:
def convert_test(df):
    df1 = df.drop('RandomFifthSentenceQuiz2', axis=1)
    df2 = df.drop('RandomFifthSentenceQuiz1',axis=1)
    df1.rename(columns={'RandomFifthSentenceQuiz1':'Quiz'}, inplace=True)
    df2.rename(columns={'RandomFifthSentenceQuiz2':'Quiz'}, inplace=True)
    
    df1["Story"] = df1['InputSentence1'].str.cat(df1['InputSentence2'])
    df1["Story"] = df1['Story'].str.cat(df1['InputSentence3'])
    df1["Story"] = df1['Story'].str.cat(df1['InputSentence4'])
    df1 = df1.drop(columns=['InputSentence1','InputSentence2','InputSentence3','InputSentence4'])
    df1 = df1[['InputStoryid','Story','Quiz','AnswerRightEnding']]
    
    df2["Story"] = df2['InputSentence1'].str.cat(df2['InputSentence2'])
    df2["Story"] = df2['Story'].str.cat(df2['InputSentence3'])
    df2["Story"] = df2['Story'].str.cat(df2['InputSentence4'])
    df2 = df2.drop(columns=['InputSentence1','InputSentence2','InputSentence3','InputSentence4'])
    df2 = df2[['InputStoryid','Story','Quiz','AnswerRightEnding']]
    
    
    return df1,df2

In [78]:
test1,test2 = convert_test(df_dev_test)

In [79]:
test1.to_csv("test1/test.tsv",sep="\t",index=None)
test2.to_csv("test2/test.tsv",sep="\t",index=None)

In [80]:
df_train['token'] = df_train['Story'].str.split(" ")

In [81]:
token = df_train['token'].values
seq_len = [len(s) for s in token]
df_train['len'] = seq_len

In [82]:
df_train.describe()

,label,len
count,2994.000000,2994.000000
mean,0.500000,32.396126
std,0.500084,6.727878
min,0.000000,14.000000
25%,0.000000,28.000000
50%,0.500000,32.000000
75%,1.000000,37.000000
max,1.000000,53.000000


In [83]:
test1

,InputStoryid,Story,Quiz,AnswerRightEnding
1497,dcb4c16a-99ae-4143-8f22-8201b6b96dad,Tony went to the lake to think about his life....,Tony decided to study algebra.,2
1498,a915b1da-42ae-4075-af49-35cc3305e982,My parents needed to buy a new television.They...,My parents bought a purple van with flowers on...,2
1499,ef5435ec-f5b7-485c-9245-dc2bd838c398,Thomas finds that coffee always tastes better ...,"After several weeks of trying, she finally suc...",1
1500,4ff62820-e78e-46bc-8ece-42d6989ad6bc,Fred was a new student in class.He was really ...,Fred was popular.,2
1501,26e62478-4a62-4f71-8449-26edcf3905c0,"One day, a dog moved into a family that had tw...",He cuddled up to the dog.,2
1502,66ae649c-a2e4-4d39-a5de-cbbe04d87381,Janel was driving to the office when she heard...,Jane changed her tire and drove to the office.,1
1503,25be43b2-b1fc-4e4a-9554-92cc58b3fa70,It was time for the baby to take a nap.The mom...,The baby then crawled around the home.,2
1504,f3cbf50b-9f9b-451f-86c6-e51a42d896fe,The office worker claimed she was the best mul...,The office worker gave up and checked into rehab.,2
1505,df30abcd-2e74-4880-b166-29f84b2e6b2b,Ty was nervous about attending summer camp not...,They all hung out at the lake that afternoon.,1
1506,c3848f83-51bf-4ebe-9452-f21d516516c8,Michael would get headaches anytime he would r...,The glasses fixed his headaches immediately.,1


In [84]:
df_train

,InputStoryid,Story,Quiz,label,token,len
971,0a3a6bc8-bc30-4cd8-bf10-b9264bc5a230,I loved to play music and need a new sound.I h...,I decided to buy a harp.,0,"[I, loved, to, play, music, and, need, a, new,...",34
1410,9a002175-1684-4db0-b719-8f2997047e07,Jamie loves to read books and has thousands of...,Jamie prefers to watch movies to reading books.,0,"[Jamie, loves, to, read, books, and, has, thou...",44
1666,aa615e4f-83c1-4b65-96a4-6a989d253a52,Jesse had just started fifth grade.He tried to...,Jesse was picked on by the other boys in school.,1,"[Jesse, had, just, started, fifth, grade.He, t...",33
234,4835c7bd-1e52-4347-b409-cf29aa542353,Evan was starting high school.He was very nerv...,Evan decided to quit high school.,0,"[Evan, was, starting, high, school.He, was, ve...",27
1417,99d14ada-400e-43bb-9b0f-35d14c9d57b7,Tori stole a candy cane from her Christmas tre...,Tori was happy her mother knew.,0,"[Tori, stole, a, candy, cane, from, her, Chris...",28
2158,52e42087-99a0-4e42-a652-87e96e27600b,Alfred was sitting alone in his room when he h...,Alfred was disappointed.,0,"[Alfred, was, sitting, alone, in, his, room, w...",29
1231,b4e4e9db-8cc5-4b39-a4c7-894ed6af9c79,My daughter has a list of things to do before ...,I want my daughter to stop being a Christian.,0,"[My, daughter, has, a, list, of, things, to, d...",35
2695,77c86c22-80e1-48ea-a5f3-07d6cf5163c9,"Me and my girlfriend went to buffalo, New york...",Next time I'll leave my girlfriend at home.,0,"[Me, and, my, girlfriend, went, to, buffalo,, ...",29
1243,65c8e152-567a-4b94-968d-48b75fb9687f,One summer my sister and I set up a lemonade s...,He bought lemonade and made us feel welcome in...,1,"[One, summer, my, sister, and, I, set, up, a, ...",38
2429,4ef6984e-daa8-4de0-9f68-b899126a7c72,Jim really wanted a new smart watch.He started...,Jim was delighted.,1,"[Jim, really, wanted, a, new, smart, watch.He,...",42


In [85]:
df

,InputStoryid,InputSentence1,InputSentence2,InputSentence3,InputSentence4,RandomFifthSentenceQuiz1,RandomFifthSentenceQuiz2,AnswerRightEnding
0,9a26ef1d-19a5-4f8b-bd04-7674f598df91,Lorraine always wanted to make biscuits the wa...,She decides to take a month away from work and...,Finally she has perfected her biscuit recipe.,She brings her mother a biscuit and her mother...,Lorraine is pleased.,Lorraine is disappointed.,1
1,2caa6eda-0fb4-4ea1-81b4-d271dd8b6b29,Larry has decided to help out his friends prep...,He needs to bring a salad to the party.,"Larry slices tomato, lettuce, and other vegeta...","When it is finished, he feels great.",Larry was happy that he could contribute to th...,Larry decided to stay home from the party.,1
2,f994b657-f673-4843-8145-2244ba93a28d,My neighbors have a problem with a mouse.,They've been chasing it for days.,They can hear it in the walls.,Today one of them saw it run across the kitche...,Tonight they will get a mousetrap to catch the...,They fed the mouse cheese and gave it mouse toys.,1
3,c85e9c3e-4c5a-4b53-8572-52f98228df2c,Francis has been inside his home for many days...,He cannot remember the last time he left his h...,Finally he puts on his boots and decides to go...,The air feels great on his face.,Francis shivers and starts walking to a park.,Francis goes inside because he hasn't been hom...,1
4,976a11ff-1a3c-4f4f-9b21-3179f2d9cf20,Amy woke up extra hungry today.,"Unfortunately, she also woke up very late.",She rushed around the house and had no time fo...,"When she made it to work, she realized her cow...",She was so grateful and promised to pay her co...,She did not want to eat.,1
5,2fed71a3-7ba4-43b1-9ebf-b15ecbabbe62,Jeff was up for a promotion in which he would ...,"He went home and studied real hard, memorizing...",He prepped for many days until he felt prepare...,He went in for his interview and passed with f...,He was pleased when he was given the promotion.,He realized he needed to quit his job.,1
6,a6e1d6d8-ddbb-4b54-a685-c642f4076e1b,Glen was in the mood for a walk.,He put on his jacket and went outside.,"While walking on the sidewalk, he tripped and ...",Glen stood back up and limped back to his house.,Glen ran his best marathon time ever.,Glen swore he'd never walk again.,2
7,17839b9c-2cb0-49f6-863b-286f2aacbbb4,"I was driving down a very unfamiliar, congeste...",The GPS directed me to a longer route on a dif...,"I gladly took the other road, but it soon beca...","The road twisted and turned, and I had no choi...",I turned the GPS off and found a better way.,It was stressful but I finally made it to my d...,2
8,e8c15b8c-9291-4e26-8b65-9daf2fc2aa22,Bertha wanted to get a haircut.,"However, she was undecided on how to cut her h...",Bertha had a idea to go to a wig store.,At the wig store Bertha tried on different wigs.,Bertha bought a bright red wig with frosted bl...,Bertha bought a jump rope.,1
9,c7150144-9538-4c8f-a84f-903484c7d0a5,I started getting into music in middle school.,"As my tastes expanded, so did my desire to att...","My friends had seen one or two, but I had not.","Finally, after a few painful Years, my time ar...",I finally saw my first concert and it was amaz...,I decided I didn't like concerts anymore.,1


In [86]:
df_train['label'].value_counts()

1    1497
0    1497
Name: label, dtype: int64